<a href="https://colab.research.google.com/github/semih108/car-price-prediction/blob/main/pricePrediction_clean_train_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
import os

# Step 1: Define the path where your CSV files are stored
csv_dir = '/content/drive/MyDrive/car_data/'

# Step 2: List all CSV files in the directory
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Step 3: Load each CSV file into a DataFrame and append it to a list
data_frames = []
for file in csv_files:
    file_path = os.path.join(csv_dir, file)
    df = pd.read_csv(file_path)
    data_frames.append(df)

# Step 4: Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Step 5: Save the combined DataFrame to a new CSV file (optional)
combined_df.to_csv('/content/drive/MyDrive/combined_car_data.csv', index=False)


In [15]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
import json

# Load Data
data = pd.read_csv('/content/drive/MyDrive/combined_car_data.csv')

# Function to find subType in the name
def find_subtype(name, make, model, all_model_details):
    name_lower = name.lower()
    model_combined = model.replace(" ", "").lower()  # Modellname ohne Leerzeichen
    if make in all_model_details:
        if model in all_model_details[make]:
            subtypes = all_model_details[make][model].keys()
            for subtype in subtypes:
                subtype_lower = subtype.lower()
                if subtype_lower in name_lower:
                    return subtype
                if model_combined == subtype_lower:  # Überprüfen, ob der zusammengesetzte Modellname ein Subtyp ist
                    return subtype
    return None

# Function to clean data
def clean_data(df):
    df['Price'] = df['Price'].str.replace(r'[^\d]', '', regex=True)
    df['Mileage'] = df['Mileage'].str.replace(r'[^\d]', '', regex=True)
    power = df['Power'].str.extract(r'(\d+) kW \((\d+) PS\)')
    df['Power_PS'] = power[1]
    df.drop(columns=['Power'], inplace=True)
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
    df['Power_PS'] = pd.to_numeric(df['Power_PS'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x if re.match(r'\d{2}/\d{4}', x) else None), format='%m/%Y', errors='coerce')
    data['Car_Age'] = data['Date'].dt.year
    data.drop(columns=['Date'], inplace=True)
    # Extract subType using the API data
    #df['subType'] = df.apply(lambda x: find_subtype(x['Name'].lower(), x['Make'].lower(), x['Model'].lower(), all_model_details), axis=1)

    # Impute missing values for subType using a placeholder
    #df['subType'] = df['subType'].fillna('unknown')
    return df


data = clean_data(data)

# Save cleaned data to a CSV file
cleaned_data_path = '/content/drive/MyDrive/cleaned_car_data.csv'
data.to_csv(cleaned_data_path, index=False)
print(f'Cleaned data saved to {cleaned_data_path}')


Cleaned data saved to /content/drive/MyDrive/cleaned_car_data.csv


In [16]:
import re
import pandas as pd
import numpy as np

# Define the function
def find_subtype(name, make, model, all_model_details):
    # Remove all spaces and replace commas with dots in the name and model
    name_lower = name.lower().replace(" ", "").replace(",", ".")

    if make.lower() in ['audi', 'volkswagen']:
        subtypes = all_model_details[make.lower()].keys()
        matching_subtype = None
        for subtype in subtypes:
            # Remove all spaces and replace commas with dots in the subtype
            subtype_lower = subtype.lower().replace(" ", "").replace(",", ".")
            # Check if the subtype is in the name
            if subtype_lower in name_lower:
                if matching_subtype is None or len(subtype_lower) > len(matching_subtype):
                    matching_subtype = subtype_lower
        return matching_subtype
    return None

# Example structure for all_model_details
all_model_details = {
    'audi': {
        '1.3': {},
        '1.6 Diesel': {},
        '1.6': {},
        '1.8': {},
        '1.8 S': {},
        '1.8 Turbo': {},
        '1.8 TFSI': {},
        '1.9': {},
        '1.9 E': {},
        '1.9 Diesel': {},
        '1.9 TDI': {},
        '1.9 TDIe': {},
        '2.0': {},
        '4.2': {},
        '2.0 Diesel': {},
        '2.0 FSI': {},
        '2.0 TDI': {},
        '2.0 TDI clean diesel': {},
        '2.0 TFSI': {},
        '2.0 TFSI E85': {},
        '2.0 TFSI g-tron': {},
        '2.0 TFSI Hybrid': {},
        '2.2': {},
        '2.2 Turbo': {},
        '2.2 ECE': {},
        '2.3': {},
        '2.4 Diesel': {},
        '2.5 TDI': {},
        '2.5 TFSI': {},
        '2.5 Turbodiesel': {},
        '2.6 V6': {},
        '2.7 V6': {},
        '2.7 V6 TDI': {},
        '2.8 V6': {},
        '2.9 V6 TFSI': {},
        '2.9 TFSI': {},
        '3.0 V6': {},
        '3.0 V6 TDI': {},
        '3.0 V6 TDI clean diesel': {},
        '3.0 V6 TFSI': {},
        '3.0 TDI': {},
        '3.0 TDI e-tron': {},
        '3.0 TFSI': {},
        '3.2 V6': {},
        '3.2 V6 FSI': {},
        '3.6 VR6 FSI': {},
        '4.0 TDI': {},
        '4.0 TFSI': {},
        '4.0 V8': {},
        '4.0 V8 TDI': {},
        '4.0 V8 TFSI': {},
        '4.2 S4 V8': {},
        '4.2 V8': {},
        '4.2 V8 FSI': {},
        '4.2 V8 TDI': {},
        '5.0 V10 FSI': {},
        '5.2 V10 FSI': {},
        '5.2 TFSI': {},
        '5.2 FSI': {},
        '5.2': {},
        '6.0 V12 TDI': {},
        'Cooper': {},
        'Cooper S': {},
        'Cooper SD': {},
        'John Cooper Works': {},
        'S4': {},
        'e-tron': {},
        'e-tron RS': {},
        'e-tron S': {},
        'e-tron35': {},
        'e-tron40': {},
        'e-tron45': {},
        'e-tron50': {},
        'e-tron55': {},
        'v6': {},
        'v8': {},
        'w12': {},
        '1.0 TFSI': {},
        '1.4 TDI': {},
        '1.4 TFSI': {},
        '1.4 TFSI ACT': {},
        '1.4 TFSI e-tron': {},
        '1.4 TGI': {},
        '1.4 TGI g-Tron': {},
        '1.5 TFSI': {},
        '1.5 TSI ACT': {},
        '1.6 TDI': {},
        '1.6 TDIe': {},
        '1.6 Turbodiesel': {},
        '2.0 TDI clean diesel': {},
        '25 TFSI': {},
        '30 g-tron': {},
        '30 TDI': {},
        '30 TFSI': {},
        '35 TDI': {},
        '35 TFSI': {},
        '40 e-tron': {},
        '40 g-tron': {},
        '40 TDI': {},
        '40 TFS': {},
        '40 TFSI': {},
        '40 TFSIe': {},
        '45 TDI': {},
        '45 TFSIe': {},
        '45 TFSI': {},
        '50 TDI': {},
        '50 TFSIe': {},
        '50 e-tron': {},
        '55 TFSI': {},
        '55 TFSIe': {},
        '55 e-tron': {},
        '60 TFSIe': {},
        '3.0tdi': {},
        '2.0tdi': {},
        '1.8tfsi': {},
        '30tdi': {},
        '35tdi': {},
        '40tdi': {},
        '40gtron': {},
        '50tdi': {},
        '55tfsi': {},
        '1.0tfsi': {},
        '1.4tfsi': {},
        '1.6tdi': {},
        '1.9tdi': {},
        '1.5tfsi': {},
        '2.7tdi': {},
        '2.0tfsi': {},
        '3.0tfsi': {},
        '4.2fsi': {},
        '4.2tdi': {},
        '40tfsi': {}
    },

    'volkswagen': {
        '2.0 TDI': {},
        '3.0 V6 TDI': {},
        '3.0 TDI': {},
        '1.5 TSI': {},
        '2.0 TSI': {},
        '1.4 eHybrid': {},
        '1.2 TSI': {},
        '1.2': {},
        '1.4 TSI': {},
        '1.6 TDI': {},
        '1.4': {},
        '1.6': {},
        '1.0': {},
        '1.6 FSI': {},
        '1.8': {},
        '1.8 Turbo': {},
        '2.0': {},
        '2.3 V5': {},
        '2.8 V6': {},
        '1.9 TDI': {},
        '1.9 SDI': {},
        '2.0 SDI': {},
        '2.0 EcoFuel': {},
        '1.6 BIFuel': {},
        '1.0 TSI': {},
        '1.4 TGI': {},
        '1.5 TSI ACT': {},
        '2.0 TFSI': {},
        '3.6 V6 FSI': {},
        'electric drive 100 kW': {},
        '0.9': {},
        '1.1': {},
        '1.3': {},
        '1.05': {},
        '2.9 VR6': {},
        '1.8 TSI': {},
        '1.5 TGI': {},
        'electric drive': {},
        '1.4 GTE': {},
        '2.0 16V': {},
        '2.5 TDI DPF': {},
        '2.0 TSI / TFSI': {},
        '3.6 V6': {},
        '1.4 TDI': {},
        '1.3 Diesel': {},
        '1.7 SDI': {},
        '2.4 Diesel': {},
        '2.5 Diesel': {},
        '2.8 TDI': {},
        '1.0 eTSI': {},
        '2.0 TDI clean diesel': {},
        '3.2 V6': {},
        '4.0 W8': {},
        '1.4 TSI EcoFuel': {},
        '1.4 TSI Hybrid': {},
        '4.0 TDI': {},
        '1.4 Diesel': {},
        '1.0 TGI': {},
        '1.4 TSI ACT': {},
        '1.0 TFSI': {},
        '1.4 TFSI': {},
        '1.4 TFSI ACT': {},
        '1.5 TFSI': {},
        '1.6 TDIe': {},
        '2.0 BiFuel': {},
        '1.2 TDI': {},
        '1.6 Turbodiesel': {},
        '2.2': {},
        '1.8 Turbo': {},
        '2.5 V6 TDI': {},
        'V6 TDI': {},
        'TDI': {},
        '4.2 FSI': {},
        '1.0 MPI': {},
        '1.2 MPI': {},
        '1.4 MPI': {},
        '1.6 MPI': {},
        '1.6 TDI CR': {},
        '2.0 TDI CR': {},
        '2.0 BiTDI': {},
        '2.5 FSI': {},
        '3.0 TSI': {},
        '4.2 V8': {},
        '4.9 V8': {},
        '5.0 V10 TDI': {},
        '6.0 W12': {}
    }
}

bmw_model = {
    '1': '1er',
    '2': '2er',
    '3': '3er',
    '4': '4er',
    '5': '5er',
    '6': '6er',
    '7': '7er',
    '8': '8er',
    'i3': 'i3',
    'i4': 'i4',
    'i7': 'i7',
    'i8': 'i8',
    'iX': 'iX',
    'iX1': 'iX1',
    'iX3': 'iX3',
    'X1': 'X1',
    'X2': 'X2',
    'X3': 'X3',
    'X4': 'X4',
    'X5': 'X5',
    'X6': 'X6',
    'X7': 'X7',
    'XM': 'XM',
    'Z1': 'Z1',
    'Z3': 'Z3',
    'Z4': 'Z4',
    'Z8': 'Z8'
}
mercedes_model = {
    '123': '123',
    '190': '190/190 E',
    'A': 'A-Klasse',
    'A-Limousine': 'A-Klasse Limousine',
    'AMG GT': 'AMG GT',
    'AMG GT 4-door': 'AMG GT 4-door',
    'B': 'B-Klasse',
    'C': 'C-Klasse',
    'C-All-Terrain': 'C-Klasse All-Terrain',
    'Citan': 'Citan',
    'CL': 'CL-Coupe',
    'CLA': 'CLA-Klasse',
    'CLC': 'CLC-Klasse',
    'CLK': 'CLK-Klasse',
    'CLS': 'CLS-Klasse',
    'E': 'E-Klasse',
    'E-All-Terrain': 'E-Klasse All-Terrain',
    'EQA': 'EQA',
    'EQB': 'EQB',
    'EQC': 'EQC',
    'EQE': 'EQE',
    'EQS': 'EQS',
    'EQV': 'EQV',
    'G': 'G-Klasse',
    'GL': 'GL-Klasse',
    'GLA': 'GLA-Klasse',
    'GLB': 'GLB-Klasse',
    'GLC': 'GLC-Klasse',
    'GLE': 'GLE-Klasse',
    'GLK': 'GLK-Klasse',
    'GLS': 'GLS-Klasse',
    'M': 'M-Klasse',
    'ML': 'M-Klasse',
    'MB': 'MB 100',
    'R': 'R-Klasse',
    'S': 'S-Klasse',
    'SL': 'SL-Klasse',
    'SLC': 'SLC-Klasse',
    'SLK': 'SLK-Klasse',
    'SLR': 'SLR McLaren',
    'SLS': 'SLS AMG',
    'Sprinter': 'Sprinter',
    'Strich Acht': 'Strich Acht',
    'T1': 'T1 Transporter',
    'T': 'T-Klasse',
    'V': 'V-Klasse',
    'V-Marco Polo': 'V-Klasse Marco Polo',
    'Vaneo': 'Vaneo',
    'Viano': 'Viano',
    'Vito': 'Vito',
    'Vito Tourer': 'Vito Tourer',
    'X': 'X-Klasse'
}

# Function to update Volkswagen Golf models based on year
def update_golf_model(row):
    if row['Make'].lower() == 'volkswagen' and 'golf' in row['Model'].lower():
        year = row['Car_Age']
        if 1974 <= year <= 1983:
            return 'Golf 1'
        elif 1983 <= year <= 1992:
            return 'Golf 2'
        elif 1991 <= year <= 1997:
            return 'Golf 3'
        elif 1997 <= year <= 2003:
            return 'Golf 4'
        elif 2003 <= year <= 2008:
            return 'Golf 5'
        elif 2008 <= year <= 2012:
            return 'Golf 6'
        elif 2012 <= year <= 2019:
            return 'Golf 7'
        elif 2019 <= year:
            return 'Golf 8'
    return row['Model']

# Load the data from the CSV file
file_path = '/content/drive/MyDrive/cleaned_car_data.csv'
data = pd.read_csv(file_path)

def determine_subtype(row):
    if row['Make'].lower() == 'bmw':
        first_char = row['Model'][0]
        return bmw_model.get(first_char, row['Model'])
    if row['Make'].lower() == 'mercedes-benz':
        first_char = row['Model'].split()[0]
        return mercedes_model.get(first_char, row['Model'])
    return row['Model']

# Update Volkswagen Golf models
data['Model'] = data.apply(update_golf_model, axis=1)

# Apply the logic to set Detected_Subtype
data['Detected_Subtype'] = np.where(
    data['Make'].str.lower().isin(['bmw', 'mercedes-benz']),
    data['Model'],
    data.apply(lambda row: find_subtype(row['Name'], row['Make'], row['Model'], all_model_details), axis=1)
)

data['Model'] = data.apply(determine_subtype, axis=1)

# Display the result
data.head(20)
data.drop(columns=['Name'], inplace=True)
data['Car_Age'] = 2024 - data['Car_Age'] # have to calculate it here because of golf model

# Save the cleaned data to a CSV file
improved_subtype = '/content/drive/MyDrive/cleaned_data_with_subtype.csv'
data.to_csv(improved_subtype, index=False)
print(f'Cleaned data saved to {improved_subtype}')


Cleaned data saved to /content/drive/MyDrive/cleaned_data_with_subtype.csv


In [17]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Load the cleaned data into a DataFrame
data = pd.read_csv('/content/drive/MyDrive/cleaned_data_with_subtype.csv')

# Shuffle the data
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

nan_sum = data.isna().sum()
print(nan_sum)

# Handle categorical data
le_transmission = LabelEncoder()
le_fuel = LabelEncoder()
le_make = LabelEncoder()
le_model = LabelEncoder()
le_subtype = LabelEncoder()

# Apply label encoding to categorical columns
data['Transmission'] = le_transmission.fit_transform(data['Transmission'].str.lower())
data['Fuel'] = le_fuel.fit_transform(data['Fuel'].str.lower())
data['Make'] = le_make.fit_transform(data['Make'].str.lower())
data['Model'] = le_model.fit_transform(data['Model'].str.lower())
data['Detected_Subtype'] = le_subtype.fit_transform(data['Detected_Subtype'].str.lower().fillna(''))

# Prepare features and target
X = data.drop(columns=['Price'])
y = data['Price']

imputer_x = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer_x.fit_transform(X), columns=X.columns)

imputer_y = SimpleImputer(strategy='median')
y = pd.Series(imputer_y.fit_transform(y.values.reshape(-1, 1)).ravel(), name='Price')

# Save the feature names
feature_names = X.columns.tolist()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Save the model and label
directory = '/content/drive/MyDrive/models'
joblib.dump(feature_names, os.path.join(directory, 'feature_names.pkl'))
joblib.dump(model, os.path.join(directory, 'car_price_predictor.pkl'))
joblib.dump(le_transmission, os.path.join(directory, 'le_transmission.pkl'))
joblib.dump(le_fuel, os.path.join(directory, 'le_fuel.pkl'))
joblib.dump(le_make, os.path.join(directory, 'le_make.pkl'))
joblib.dump(le_model, os.path.join(directory, 'le_model.pkl'))
joblib.dump(le_subtype, os.path.join(directory, 'le_subtype.pkl'))


Make                   0
Model                  0
Price                718
Mileage               75
Transmission           0
Fuel                   0
Power_PS              13
Car_Age              591
Detected_Subtype    4174
dtype: int64
Mean Absolute Error: 5052.193977286701


['/content/drive/MyDrive/models/le_subtype.pkl']

In [18]:
import numpy as np
import joblib
import pandas as pd

# Load the model, label encoders, and feature names
model = joblib.load('/content/drive/MyDrive/models/car_price_predictor.pkl')
le_transmission = joblib.load('/content/drive/MyDrive/models/le_transmission.pkl')
le_fuel = joblib.load('/content/drive/MyDrive/models/le_fuel.pkl')
le_make = joblib.load('/content/drive/MyDrive/models/le_make.pkl')
le_model = joblib.load('/content/drive/MyDrive/models/le_model.pkl')
le_subtype = joblib.load('/content/drive/MyDrive/models/le_subtype.pkl')
feature_names = joblib.load('/content/drive/MyDrive/models/feature_names.pkl')

# Function to check for unseen labels
def check_label(encoder, label):
    label_lower = label.lower()
    if label_lower not in [cls.lower() for cls in encoder.classes_]:
        raise ValueError(f"Label '{label}' not found in encoder classes: {encoder.classes_}")
    return encoder.transform([label_lower])[0]

# Function to predict car price
def predict_car_price(make, model_name, mileage, transmission, fuel, power_ps, car_age, subtype):
    # Check and transform categorical inputs
    make_encoded = check_label(le_make, make)
    model_encoded = check_label(le_model, model_name)
    transmission_encoded = check_label(le_transmission, transmission)
    fuel_encoded = check_label(le_fuel, fuel)
    subtype_encoded = check_label(le_subtype, subtype)

    # Prepare the feature array with correct column names and order
    features = pd.DataFrame([[make_encoded, model_encoded, mileage, transmission_encoded, fuel_encoded, power_ps, car_age, subtype_encoded]], columns=feature_names)

    # Debug: Print the features to check correctness
    print("Features for prediction:")
    print(features)

    # Predict the price
    predicted_price = model.predict(features)[0]

    return predicted_price

# Example usage
try:
    make = 'Audi'  # example brand
    model_name = 'q3'  # example model
    mileage = 80000  # example mileage (corrected unrealistic value)
    transmission = 'schaltgetriebe'  # example transmission
    fuel = 'diesel'  # example fuel
    power_ps = 140  # example power in PS
    car_age = 10  # example car age
    subtype = '2.0tdi'

    predicted_car_price = predict_car_price(make, model_name, mileage, transmission, fuel, power_ps, car_age, subtype)
    print(f'The predicted price of the car is: € {predicted_car_price:.2f}')
except ValueError as e:
    print(f'Error: {e}')


Features for prediction:
   Make  Model  Mileage  Transmission  Fuel  Power_PS  Car_Age  \
0     0    118    80000             3     3       140       10   

   Detected_Subtype  
0                51  
The predicted price of the car is: € 16741.66
